In [1]:
# everything from here on is copied or modified from etc_code.py

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn


In [2]:
os.environ['ATLAS_ETC_PATH']="/Users/nanatang/Documents/GitHub/stsci_etc/Spectroscopic_ETC/atlas_etc-master"


In [3]:
###### HELPER FUNCTIONS #######

## Read parameter file
# These files have the "SExtractor-type" format (key val comment)
# Returns a dictionary with key and value
def read_par_file(filename):
    '''
    This function reads the telescope parameter file and outputs a dictionary with (key,value). \\
    USAGE: read_par_file(filename) where filename is the name of the parameter file to read
    '''
    with open(filename , "r") as f:
        lines = f.readlines()
    

    # replace tab and line break
    lines = [line.replace("\t","  ").replace("\n", "") for line in lines ] 

    # get lines that are not empty or commented out
    lines = [ line for line in lines if line != "" if line[0] != "#" ] 


    # extract key, val, comment (if exist)
    extract = dict()
    for line in lines:
        try:
            key = line.split()[0]
            val = line.split()[1]
            try: # check if the value can be interpreted as float
                val = float(line.split()[1])
            except: # if not, make it a string
                val = str(line.split()[1])
            extract[key] = val
        except:
            print("Cannot interpret/read the line %s" & line)
            quit()
    
    return(extract)


In [4]:
def degrade_resolution(in_wave, in_flux , center_wave , R, disp, tot_Npix):
    '''
    Degrades an input array to the telescope resolution. \\ 
    USAGE: degrade_resolution(in_wave , in_flux , center_wave , R , disp) where \\
        in_wave: wavelength \\
        in_flux: flux or transmission \\
        center_wave: center wavelength \\
        R: Spectral resolution \\
        disp: dispersion [angstrom/px] \\
    OUTPUT: Dictionary including the degraded wavelength vs. flux: (lam,flux)
    '''
# basically what this does is it takes wavelengths (x axis) and flux (y axis) and converts it to velocity (through
# interpolation etc) and then it convolves it with a gaussian kernel (I guess to preserve the shape of the peaks? not sure)
# and then it converts everything back to wavelength and to px

    # Number of pixels to be output - 50%
    # more than are on the detector to
    # cover the K band
    Npix_spec=tot_Npix * 3./2.
        # specifically for K band, allowing more pixels for this band

    #the speed of light in cm/s
    c=np.log10(29979245800.)
       # numerical convenience

    # make a "velocity" grid centered at
    # the central wavelength of the band
    # sampled at 1 km/s
    vel=(np.arange(600001)-300000) #ok
    in_vel=(in_wave/center_wave-1)*10.**(1*c-5) # but why
        # convert wavelength to relative velocity

    # create vectors in velocity space
    in_vel_short = in_vel[ np.where( (in_vel > vel[0]) & (in_vel < vel[600000]) )[0] ]
    in_flux_short = in_flux[ np.where( (in_vel > vel[0]) & (in_vel < vel[600000]) )[0] ]
        # picking out realistic values (keeping good indices)
        
    #interp_flux = np.interp(vel, in_flux_short, in_vel_short)
    interp_flux = np.interp(vel, in_vel_short, in_flux_short)
        # interpolate to 600000 points

    #sigma  = the resolution of the spectrograph
    sigma = (10.**(c-5)/R)/(2*np.sqrt(2*np.log(2)))
        # convert to velocity resolution, potentially res per px
        # *****this part is important but idk why, try to figure it out


    # make a smaller velocity array with
    # the same "resolution" as the steps in
    # vel, above
    n = round(8.*sigma)
        # (potentially) making a kernel that is 8px 
    if (n % 2 == 0):
        n = n + 1
    vel_kernel = np.arange(n) - np.floor(n/2.0)

    # a gaussian of unit area and width sigma
    gauss_kernel = (1/(np.sqrt(2*np.pi)*sigma)) * np.exp(-0.5*vel_kernel**2.0/sigma**2.0)
        # shape the kernel
        # look up the equation for gaussian kernal and figure out the significance of sigma used here
        # like how does resolution shape/define the kernel

    # convolve flux with gaussian kernel
    convol_flux = np.convolve(interp_flux, gauss_kernel , mode="same") 
        # convolve moving kernel
    convol_wave = center_wave * (vel*10.**(-1*c+5.0) + 1.0 )
        # convert back to wavelength

    # and the real pixel scale of mosfire
    real_wave = np.arange(Npix_spec) * disp * 10.**(-4.)
    real_wave = real_wave - real_wave[int(np.round(Npix_spec/2.))]   
    real_wave = real_wave + center_wave 
        # wavelength to px

    # interpolate onto the pixel scale of the detector
    out_wave = real_wave
    out_flux = np.interp(real_wave , convol_wave, convol_flux)
        # interpolating to number of px (b/c working from km/px or lam/px)
    
    out = {"lam": out_wave,
          "flux": out_flux}
    
    return(out)



In [14]:
#### MAIN PROGRAM ########


### ============= DIRECTORY STRUCTURE AND NAMES ======================

## Directory structure
# Make sure to have the environment variable "ATLAS_ETC_PATH" set.
try:
    if os.environ["ATLAS_ETC_PATH"] == "none":
        pass
except:
    print("Environment variable 'ATLAS_ETC_PATH' does not seem to be set. Add it to your shell initialization script and try again. See manual for more details.")
    quit()

DIRS = dict()
DIRS["bk_path"] = "%s/etc_data/background/" % os.environ["ATLAS_ETC_PATH"] # path to background data files
DIRS["tp_path"] = "%s/etc_data/spec_throughput/" % os.environ["ATLAS_ETC_PATH"] # path to spectral throughput data files
DIRS["filt_path"] = "%s/etc_data/filter/" % os.environ["ATLAS_ETC_PATH"] # path to filter transmission data files
DIRS["tel_path"] = "%s/etc_data/telescope/" % os.environ["ATLAS_ETC_PATH"] # path to telescope data files




### ============= TELESCOPE DEFINITIONS AND OTHER CONSTANTS ======================
## The file should contain
# - linearity limits
# - telescope stats
# - other stats (mirrors reflectivity, dark current, etc)
userinput = {"band": "NIR",
             "slit_width":2.8}
try:
    telstats = read_par_file(os.path.join(DIRS["tel_path"] , "telescope_data_%s.txt" % userinput["band"]) )
except:
    print("Something went wrong when reading the telescope parameter file. Check file name!")
    quit()



# Add R-theta product: multiply by slit width to get resolution
telstats["rt"] = 1000 * telstats["slit_W"] 
        # intrinsic resolution, adding to the telescope list
        # 3


## Load Filters Transmission Curve ---------------
try:
    filter_trans = ascii.read(os.path.join(DIRS["filt_path"],telstats["filt_filename"]),names=["lam","trans"])
except:
    print("Something went wrong when reading the filter transmission file. Check file name!")
    quit()     


## Load Spectral Transmission Curve --------------
try:
    throughput = ascii.read(os.path.join(DIRS["tp_path"] , telstats["tp_filename"]) , names=["tp_wave","throughput"] )
except:
    print("Something went wrong when loading the throughput curve. Check file name!")
    quit()

#throughput["throughput"] = throughput["throughput"] * telstats["Mref"]**telstats["Nmirror"]

'''
throughput = [[1, 2], [3, 4]]
for row in throughput:
    for item in row:
    row[1] = row[1] * 2
    
throughput == [2, 4, 6]
'''
    

## General Constants ---------------
speedoflight = np.log10(29979245800) # speed of light in cm/2
hplank = np.log10(6.626068)-27 # Plank constant in erg*s
loghc = speedoflight + hplank # H*c in log(erg * cm)
f_nu_AB = 48.59

R = telstats["rt"] / userinput["slit_width"]

## Sky Constants ---------------

# Sky transparency
# Not used since in space

# Sky Background
# lam is in micron
# Bkg are in MJy/sr and need to be converted to photons/s/arcsec2/nm/m2
try:
    skybackground = ascii.read(os.path.join(DIRS["bk_path"] , telstats["bk_filename"]) , names=["lam","lowBkg","midBkg","highBkg"] )
except:
    print("Something went wrong when reading the sky background file. Check file name!")
    quit()

bkg = skybackground["midBkg"].copy()
bkg = bkg / 206265**2 # from MJy/sr to MJy/arcsec2
bkg = bkg * 1e6 # from MJy/arcsec2 to Jy/arcsec2
bkg = bkg * 1e-26 # from Jy/arcsec2 to W/m2/s/Hz/arcsec2
bkg = bkg * 1e7 # from W/m2/s/Hz/arcsec2 to erg/m2/s/Hz/arcsec2
bkg = bkg / (10**loghc / (skybackground["lam"]*1e-4)) # from erg/m2/s/Hz/arcsec2 to ph/m2/s/Hz/arcsec2
bkg = bkg * 10**speedoflight / (skybackground["lam"]*1e-4)**2 # from ph/m2/s/Hz/arcsec2 to ph/m2/s/cm/arcsec2
bkg = bkg * 1e-7 # from ph/m2/s/cm/arcsec2 to phm2/s/nm/arcsec2
skybackground["Bkg"] = bkg.copy()


filter_trans_convolved = degrade_resolution(in_wave=filter_trans["lam"],
                        in_flux=filter_trans["trans"],
                        center_wave=telstats["lambda"],
                        R=R,
                        disp=telstats["disp"],
                        tot_Npix=telstats["tot_Npix"]
                        )
filter_trans_convolved["fltSpecObs"] = filter_trans_convolved.pop("flux")
filter_trans_convolved["wave_grid"] = filter_trans_convolved.pop("lam")

# the relevant portion of the spectrum
band_index = np.where(filter_trans_convolved["fltSpecObs"] > 0.1)[0] # this is always the same for the degrade_resolution output

skybackground_convolved = degrade_resolution(in_wave=skybackground["lam"],
                        in_flux=skybackground["Bkg"],
                        center_wave=telstats["lambda"],
                        R=R,
                        disp=telstats["disp"],
                        tot_Npix=telstats["tot_Npix"]
                        )
skybackground_convolved["raw_bkSpecObs"] = skybackground_convolved.pop("flux")
skybackground_convolved["wave_grid"] = skybackground_convolved.pop("lam")
sel_zero = np.where( skybackground_convolved["raw_bkSpecObs"] < 0)[0]
#sel_nonzero = np.where( (skybackground_convolved["raw_bkSpecObs"] > 0) & (skybackground_convolved["raw_bkSpecObs"] != -0) )[0]

skybackground_convolved["raw_bkSpecObs"][sel_zero] = 0

skybackground_convolved["raw_bkSpecObs"] = skybackground_convolved["raw_bkSpecObs"][band_index]
skybackground_convolved["wave_grid"] = skybackground_convolved["wave_grid"][band_index]

# this is the final wave grid to be used in the following
wave_grid = skybackground_convolved["wave_grid"].copy()


center = 6563 * (1+1.5)

# resolution at the central wavelength
res = center / R

# the width of the spectral line before going through the spectrograph
real_width = center * 30 * 10**(-speedoflight + 5)

# the line width in microns that should be observed
width = np.sqrt(real_width**2 + res**2)


line_index = np.where( np.abs(wave_grid - center) <= 0.5*width)[0]

print(0.5*width)
print(width)

8.244723243569016
16.489446487138032
